In [1]:
import glob
import satpy
import xarray

import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from functools import partial
import traceback

import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

import sys
sys.path.append('../dataproc/')
from utils import *

In [3]:
abbvs = {
    "nio": "North Indian Ocean"
}

In [4]:
def get_satmaps(region, name):
    
    ERA5_BASE_DIR = "/vol/bitbucket/pn222/satellite/era5"
    
    if region == "nio":
        map_x0, map_y0 = 78.662109, 20.344627 ; hs_length = 20
        if name == "kyarr":
            map_x0, map_y0 = 65.662109, 20.344627 ; hs_length = 20
        if name == "gulab-shaheen":
            map_x0, map_y0 = 72.662109, 20.344627 ; hs_length = 23
        
    map_bounds = get_bbox_square(map_x0, map_y0, hs_length)
    era5_nc_files = sorted(glob.glob(f'{ERA5_BASE_DIR}/data/nc/{name}/*.nc'))
    mc_era5, era5_map_bounds =  get_era5_map(era5_nc_files, map_bounds)

    satmaps = {
        "region": region, 
        "name": name,
        "map_bounds": [float(x) for x in era5_map_bounds],
        "era5_fns": era5_nc_files
    }
    satmaps["satmaps"] = []
    
    if region == "nio":
        IR108_BASE_DIR = "/vol/bitbucket/pn222/satellite/mosdac"
        h5_files = sorted(glob.glob(f"{IR108_BASE_DIR}/data/h5/{name}/*/*.h5"))
        for h5_file in h5_files[len(h5_files)//2:len(h5_files)//2+5]:
            date = " ".join(h5_file.split('/')[-1].split('_')[1:3])
            date = datetime.strptime(date, "%d%b%Y %H%M")
            satmaps["satmaps"].append({"date": date, "ir108_fn": h5_file})    

    hrs = np.array([np64_to_datetime(x.values) for x in mc_era5["time"]])
    for idx in range(len(satmaps["satmaps"])):
        try:
            era5_idx = np.where(hrs == satmaps["satmaps"][idx]["date"])[0][0]
            satmaps["satmaps"][idx]["era5_idx"] = era5_idx
        except Exception as e:            
            print(f"[{name.upper()}]: Processing error at {satmaps['satmaps'][idx]['date']}")
            print(traceback.format_exc())
            del satmaps["satmaps"][idx]

    satmaps["count"] = len(satmaps["satmaps"])

    return satmaps

In [5]:
region = "nio" ; name = "gulab-shaheen"
satmaps = get_satmaps(region, name)

In [6]:
GFS_VARIABLES = [
    ("u10", "10m_u_component_of_wind", "10m Wind U Component"),
    ("v10", "10m_v_component_of_wind", "10m Wind V Component"),
    ("tp", "total_precipitation", "Total Precipitation"),
    ("tcc", "total_cloud_cover", "Total Cloud Cover"),
]

In [7]:
def get_map_img(m, ax, data, map_bounds,
                x=None, y=None, contour=False,
                title=None,
                y_labels=[1, 0, 0, 0],
                x_labels=[0, 0, 0, 1], colorbar=False):
    
    kwargs = {
        "linewidth": 0.5,
        "color": "k",
        "ax": ax
    }

    if colorbar:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.1)
        
    m.drawcoastlines(**kwargs)
    m.drawcountries(**kwargs)
    m.drawstates(**kwargs)

    if contour: im = m.contourf(x, y, data, cmap='jet', ax=ax)
    else: im = m.imshow(data, origin='upper', extent=map_bounds, cmap="gray", ax=ax)

    m.drawparallels(range(int(map_bounds[1])-1, int(map_bounds[3]), 7), labels=y_labels, fontsize=10, ax=ax)
    m.drawmeridians(range(int(map_bounds[0])-1, int(map_bounds[2]), 7), labels=x_labels, fontsize=10, ax=ax)

    if title: ax.set_title(title)

    if colorbar:
        fig.colorbar(im, cax=cax, orientation='vertical')    

    return im

def update(frame_idx, era5):
    fig.clear()
    axs = fig.subplot_mosaic([['main', 'era5_0', 'era5_1'],
                          ['main', 'era5_2', 'era5_3']],
                          gridspec_kw={'width_ratios':[2, 1, 1]})
    ir108_metadata = satmaps['satmaps'][frame_idx]
    if region == "nio":
        ir108_scn = get_insat3d_ir108_scn(ir108_metadata['ir108_fn'], satmaps['map_bounds'])

    get_map_img(m, axs['main'], ir108_scn.data, satmaps['map_bounds'])
    axs['main'].set_title(f"Cyclone {name.replace('-', ' ').title()} - {abbvs[region]}\nIR 10.8 µm\n{ir108_metadata['date'].strftime('%Y-%m-%d %H:%M')}")
    
    for idx, variable in enumerate(GFS_VARIABLES):
        ax = axs[f'era5_{idx}']
        
        x, y = lon_grid, lat_grid
        data = era5.variables[variable[0]][:][ir108_metadata['era5_idx']]
        cf = get_map_img(m, ax, data, satmaps['map_bounds'],
                         lon_grid, lat_grid, 
                         y_labels=[0, 0, 0, 0],
                         x_labels=[0, 0, 0, 0],
                         contour=True)
        
        ax.set_title(f"{variable[2]}")

In [8]:
era5, _ = get_era5_map(satmaps['era5_fns'], map_bounds=satmaps['map_bounds'])
m = Basemap(llcrnrlon=satmaps['map_bounds'][0], llcrnrlat=satmaps['map_bounds'][1],
            urcrnrlon=satmaps['map_bounds'][2], urcrnrlat=satmaps['map_bounds'][3],
            projection='cyl', resolution='l')

lat = era5.variables["latitude"][:]
lon = era5.variables["longitude"][:]
lon_grid, lat_grid = np.meshgrid(lon, lat)

update = partial(update, era5=era5)

fig = plt.figure(figsize=(11.15,6), constrained_layout=True)
animation = FuncAnimation(fig, update, frames=5, interval=250)
# animation.save(f'./gifs/{region}/{name}_ir108_era5.gif', writer='imagemagick')

plt.close()  
HTML(animation.to_jshtml())